In [1]:
import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()

In [2]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cv2.destroyAllWindows()
cap.release()

In [3]:
face_features = extract_image_features(frame)
# show_extraction_results(face_features)

In [21]:
img_size = 64
n_channel = 1
mask_size = 25

conv1_eye_size = 9
conv1_eye_out = 64
conv1_eye_strides = 2
pool1_eye_size = 2
pool1_eye_stride = 2

conv2_eye_size = 5
conv2_eye_out = 64
pool2_eye_size = 2
pool2_eye_stride = 2

conv3_eye_size = 3
conv3_eye_out = 64
pool3_eye_size = 2
pool3_eye_stride = 2

# conv4_eye_size = 1
# conv4_eye_out = 64
# pool4_eye_size = 2
# pool4_eye_stride = 2

conv1_face_size = 9
conv1_face_out = 64
conv1_face_strides = 2
pool1_face_size = 2
pool1_face_stride = 2

conv2_face_size = 5
conv2_face_out = 64
pool2_face_size = 2
pool2_face_stride = 2

conv3_face_size = 3
conv3_face_out = 64
pool3_face_size = 2
pool3_face_stride = 2

# conv4_face_size = 1
# conv4_face_out = 64
# pool4_face_size = 2
# pool4_face_stride = 2

fc_eye_size = 128
fc_face_size = 128
fc2_face_size = 64
fc_face_mask_size = 128
fc2_face_mask_size = 128
fc_size = 128
fc2_size = 2

from keras.layers import *
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
# from src.model_config import *

import numpy as np

right_input = Input(shape=(img_size, img_size, n_channel))
left_input = Input(shape=(img_size, img_size, n_channel))
face_input = Input(shape=(img_size, img_size, n_channel))
mask_input = Input(shape=(mask_size * mask_size,))

conv1_right = Conv2D(conv1_eye_out, kernel_size=conv1_eye_size, strides=conv1_eye_strides)(right_input)
conv1_pool_right = MaxPooling2D(pool_size=pool1_eye_size, strides=pool1_eye_stride)(conv1_right)
conv2_right = Conv2D(conv2_eye_out, kernel_size=conv2_eye_size)(conv1_pool_right)
conv2_pool_right = MaxPooling2D(pool_size=pool2_eye_size, strides=pool2_eye_stride)(conv2_right)
conv3_right = Conv2D(conv3_eye_out, kernel_size=conv3_eye_size)(conv2_pool_right)
conv3_pool_right = MaxPooling2D(pool_size=pool3_eye_size, strides=pool3_eye_stride)(conv3_right)
# conv4_right = Conv2D(conv4_eye_out, kernel_size=conv4_eye_size)(conv3_pool_right)
# conv4_pool_right = MaxPooling2D(pool_size=pool4_eye_size, strides=pool4_eye_stride)(conv4_right)

conv1_left = Conv2D(conv1_eye_out, kernel_size=conv1_eye_size, strides=conv1_eye_strides)(left_input)
conv1_pool_left = MaxPooling2D(pool_size=pool1_eye_size, strides=pool1_eye_stride)(conv1_left)
conv2_left = Conv2D(conv2_eye_out, kernel_size=conv2_eye_size)(conv1_pool_left)
conv2_pool_left = MaxPooling2D(pool_size=pool2_eye_size, strides=pool2_eye_stride)(conv2_left)
conv3_left = Conv2D(conv3_eye_out, kernel_size=conv3_eye_size)(conv2_pool_left)
conv3_pool_left = MaxPooling2D(pool_size=pool3_eye_size, strides=pool3_eye_stride)(conv3_left)
# conv4_left = Conv2D(conv4_eye_out, kernel_size=conv4_eye_size)(conv3_pool_left)
# conv4_pool_left = MaxPooling2D(pool_size=pool4_eye_size, strides=pool4_eye_stride)(conv4_left)

conv1_face = Conv2D(conv1_eye_out, kernel_size=conv1_eye_size, strides=conv1_face_strides)(face_input)
conv1_pool_face = MaxPooling2D(pool_size=pool1_eye_size, strides=pool1_eye_stride)(conv1_face)
conv2_face = Conv2D(conv2_eye_out, kernel_size=conv2_eye_size)(conv1_pool_face)
conv2_pool_face = MaxPooling2D(pool_size=pool2_eye_size, strides=pool2_eye_stride)(conv2_face)
conv3_face = Conv2D(conv3_eye_out, kernel_size=conv3_eye_size)(conv2_pool_face)
conv3_pool_face = MaxPooling2D(pool_size=pool3_eye_size, strides=pool3_eye_stride)(conv3_face)
# conv4_face = Conv2D(conv4_eye_out, kernel_size=conv4_eye_size)(conv3_pool_face)
# conv4_pool_face = MaxPooling2D(pool_size=pool4_eye_size, strides=pool4_eye_stride)(conv4_face)

right_out = Reshape([-1, int(np.prod(conv3_pool_right.get_shape()[1:]))])(conv3_pool_right)
left_out = Reshape([-1, int(np.prod(conv3_pool_left.get_shape()[1:]))])(conv3_pool_left)

eyes = Concatenate(axis=-1)([right_out, left_out])
eyes = Dense(fc_eye_size, activation="relu")(eyes)
eyes = Flatten()(eyes)

face = Reshape([-1, int(np.prod(conv3_pool_face.get_shape()[1:]))])(conv3_pool_face)
face = Dense(fc_face_size, activation="relu")(face)
face = Dense(fc2_face_size, activation="relu")(face)
face = Flatten()(face)

face_mask = Dense(fc_face_mask_size, activation="relu")(mask_input)
face_mask = Dense(fc2_face_mask_size, activation="relu")(face_mask)

fc = Concatenate(axis=-1)([eyes, face, face_mask])
fc2 = Dense(fc_size, activation="relu")(fc)
out = Dense(fc2_size, activation="sigmoid")(fc2)

model = Model([left_input, right_input, face_input, mask_input], out)
model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_51 (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_102 (Conv2D)             (None, 28, 28, 64)   5248        input_49[0][0]                   
______________________________________________________________________________________

In [11]:
from src.model import model

model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 54, 54, 32)   11648       input_1[0][0]                    
_______________________________________________________________________________________

In [4]:
X_left, X_right, X_face, X_mask, Y = [], [], [], [], []

In [ ]:
cap = cv2.VideoCapture(0)
# frame_width, frame_height = 320, 240
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass


x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(50):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 2000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                face_features = extract_image_features(frame)
                try:
                    face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
                    X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_mask.append(face_features[2])
                    Y.append([xc,yc])
                except Exception as e:
                    print(e)
#                 img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                 X_temp.append(np.expand_dims(img, -1))
                
#                 Y.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [271]:
from ipywidgets import IntProgress
from IPython.display import display

target_height, target_width = 32, 64
max_count = len(X_temp)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

X_left, X_right, X_face, X_mask, Y = [], [], [], [], []
for i, frame in enumerate(X_temp):
    face_features = extract_image_features(frame)
    try:
        face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
        X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_mask.append(face_features[2])
        Y.append(Y_temp[i])
    except:
        pass
    f.value += 1

IntProgress(value=0, max=2100)

In [31]:
X_left = np.expand_dims(np.array(X_left), -1)
X_right = np.expand_dims(np.array(X_right), -1)
X_face = np.expand_dims(np.array(X_face), -1)
X_mask = np.array(X_mask)
Y = np.array(Y)
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

In [277]:
X_left = np.concatenate([X_left, X_left_old])
X_right = np.concatenate([X_right, X_right_old])
X_face = np.concatenate([X_face, X_face_old])
X_mask = np.concatenate([X_mask, X_mask_old])
Y = np.concatenate([Y, Y_old])

In [257]:
X_left_old, X_right_old, X_face_old, X_mask_old, Y_old = X_left, X_right, X_face, X_mask, Y
X_left, X_right, X_face, X_mask, Y = [],[],[],[],[]

In [34]:
import pickle

pickle.dump([X_left, X_right, X_face, X_mask, Y], open("save2_c1.pkl", "wb"))

In [193]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

In [133]:
X_train = [X_left[train_set], X_right[train_set], X_face[train_set], X_mask[train_set]]
X_test = [X_left[test_set], X_right[test_set], X_face[test_set], X_mask[test_set]]

In [ ]:
model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=100)

In [2]:
import pickle

[X_left, X_right, X_face, X_mask, Y] = pickle.load(open("save.pkl", "rb"))
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

(9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 625) (9746, 2)


In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

X_train = [X_left[train_set], X_right[train_set], X_face[train_set], X_mask[train_set]]
X_test = [X_left[test_set], X_right[test_set], X_face[test_set], X_mask[test_set]]

In [4]:
from keras.models import load_model
model = load_model("checkpoints")

In [22]:
history = model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=200, validation_data=(X_test, np.array(Y)[test_set]))

Epoch 1/200
35/35 [==============================] - 8s 235ms/step - loss: 0.0472 - val_loss: 0.0375
Epoch 2/200
35/35 [==============================] - 7s 198ms/step - loss: 0.0353 - val_loss: 0.0342
Epoch 3/200
35/35 [==============================] - 7s 193ms/step - loss: 0.0313 - val_loss: 0.0283
Epoch 4/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0264 - val_loss: 0.0238
Epoch 5/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0224 - val_loss: 0.0236
Epoch 6/200
35/35 [==============================] - 7s 187ms/step - loss: 0.0176 - val_loss: 0.0168
Epoch 7/200
35/35 [==============================] - 7s 192ms/step - loss: 0.0134 - val_loss: 0.0123
Epoch 8/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0103 - val_loss: 0.0088
Epoch 9/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0081 - val_loss: 0.0082
Epoch 10/200
35/35 [==============================] - 7s 199ms/step - loss: 0.0073 - val_lo

35/35 [==============================] - 7s 188ms/step - loss: 3.1116e-04 - val_loss: 0.0013
Epoch 80/200
35/35 [==============================] - 7s 193ms/step - loss: 2.7322e-04 - val_loss: 0.0014
Epoch 81/200
35/35 [==============================] - 7s 189ms/step - loss: 3.0122e-04 - val_loss: 0.0014
Epoch 82/200
35/35 [==============================] - 7s 190ms/step - loss: 3.5340e-04 - val_loss: 0.0014
Epoch 83/200
35/35 [==============================] - 6s 182ms/step - loss: 3.8844e-04 - val_loss: 0.0016
Epoch 84/200
35/35 [==============================] - 7s 187ms/step - loss: 3.8375e-04 - val_loss: 0.0015
Epoch 85/200
35/35 [==============================] - 6s 183ms/step - loss: 3.2429e-04 - val_loss: 0.0015
Epoch 86/200
35/35 [==============================] - 7s 186ms/step - loss: 3.0548e-04 - val_loss: 0.0013
Epoch 87/200
35/35 [==============================] - 7s 190ms/step - loss: 2.4974e-04 - val_loss: 0.0014
Epoch 88/200
35/35 [==============================] - 7s 19

KeyboardInterrupt: 

In [23]:
model.predict(X_train)

array([[0.05270752, 0.459716  ],
       [0.9177382 , 0.7912232 ],
       [0.6546875 , 0.49460116],
       ...,
       [0.17606658, 0.4483772 ],
       [0.5178365 , 0.40882057],
       [0.26228034, 0.37397596]], dtype=float32)

In [25]:
history

import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()
import pickle

[X_left, X_right, X_face, X_mask, Y] = pickle.load(open("save.pkl", "rb"))
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

X_train = [X_left[train_set], X_right[train_set], X_face[train_set], X_mask[train_set]]
X_test = [X_left[test_set], X_right[test_set], X_face[test_set], X_mask[test_set]]
from keras.models import load_model
model = load_model("checkpoints")
history = model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=200, validation_data=(X_test, np.array(Y)[test_set]))
from src.model import model

model.compile(optimizer='adam', loss="mse")
model.summary()
img_size = 64
n_channel

In [24]:
# Plot history: MAE
plt.figure(figsize=(10,6), dpi=80)
plt.plot(history.history['loss'], label='MSE (training data)')
plt.plot(history.history['val_loss'], label='MSE (validation data)')
plt.title('MSE over training')
plt.ylabel('MSE value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

NameError: name 'history' is not defined

<Figure size 800x480 with 0 Axes>